# Verilog Overlay Tutorial

This is a tutorial targeted at Verilog and Python users who wish to use Zynq 7000 to do Verilog FPGA development and interface their Verilog in the PL to the PS system.

This tutorial will use Verilog to develop a Simple Logic Module. Code and notes are available at <a href=http://github.com/rogerpease/PYNQPublicRepository/SimpleLogicModule>http://github.com/rogerpease/PYNQPublicRepository/SimpleLogicModule</A>

This tutorial was developed and tested with Viviado 2019.1 and a <A HREF=../getting_started/pynq_z2_setup.html>PYNQ-Z2</A> board. 

## Design Steps and decisions

This tutorial will follow these design steps:

1. Develop Base IP for the PL. The PL will be interfaced to the PS through memory mapping.    
2. Instance the IP in an AXI IP Wrapper. The AXI IP makes your design memory map accessible by the CPU. 
3. Instance the AXI-wrapped IP in an FPGA level project. These will be used to build the bitstream and hwh files.

## Developing Base IP

For this first example a simple combinational logic module is used. There is a peripheral-level clock and reset available or the Clocking Wizard may be used to provide faster or slower clocks a peripheral if clocking is desired. 

```
   module SimpleLogicModule (
      input [31:0] a,
      input [31:0] b,
      output [31:0] sum,
      output [31:0] difference,
      output [31:0] bitwiseAnd,
      output [31:0] bitwiseOr,
      output [31:0] bitwiseXor,
      output [31:0] bitwiseXNor
      );
      
     assign sum = a+b;
     assign difference = a-b;
     assign bitwiseAnd = a & b;
     assign bitwiseOr  = a | b;
     assign bitwiseXor  = a ^  b;
     assign bitwiseXNor = a ~^ b;
   endmodule
```



## Instancing your IP inside an AXI Peripheral

### Develop your IO Port to Memory Map

IO Ports used to supply or receive data must be mapped to register offsets. A peripheral clock and reset are available but not used in this tutorial. 

| IO Port | Register Offset | Register Name |
| :- | :- | :- |
|   a   | 0x00           | slv_reg0      |
|   b   | 0x04           | slv_reg1      |
| sum   | 0x08           | slv_reg2      |
| difference | 0x0C      | slv_reg3      |
| and   | 0x10           | slv_reg4      |
| or    | 0x14           | slv_reg5      |
| xor   | 0x18           | slv_reg6      |
| xnor  | 0x1C           | slv_reg7      |


### Building AXI IP

Now a AXI4 IP wrapper must be developed around the IP we are instancing. 

* Create a new project called "SimpleLogicModuleAXIIP". 
* Select "Tools"->"Create and Package New IP". 
* Select "Create a new AXI4 Peripheral" and fill in naming information.
    - Name: SimpleLogicModuleAXIIP
    - Version: 1.0 
    - Use default Display Name
    - Description and IP location are user's choice but note the location of the ip_repo as it will be necessary for the next step. 
* Click "Next" and the tool will ask to configure the AXI interface. 
    - Interface type: Lite
    - Interface mode: Slave 
    - Data Width: 32 bits
    - Number of registers: 8 
* Click "Next" and select "Edit IP". Click "Finish".


### Instancing your IP

The tool will generate a skeleton AXI4 peripheral which can be updated to suit the needs of peripheral. 

![AXI4 Generated Peripheral Module Wrapper](../images/axi4_module_design_source.png) 


#### Updating the AXI IP

In the SimpleLogicModuleAXIIP_v1_0_S00_AXI_inst file, at the bottom, add the SimpleLogicModule instance in between the User Logic lines along with wires for any outputs (the inputs you can pull directly from slv_reg0/1.

```Verilog
    // Add user logic here
    wire [31:0] sum;
    wire [31:0] difference;
    wire [31:0] bitwiseAnd;
    wire [31:0] bitwiseOr;
    wire [31:0] bitwiseXor;
    wire [31:0] bitwiseXNor;
    
    SimpleLogicModule SimpleLogicModule_inst(
     .a(slv_reg0),
     .b(slv_reg1),
     .sum(sum),
     .difference(difference),
     .bitwiseAnd(bitwiseAnd),
     .bitwiseOr(bitwiseOr),
     .bitwiseXor(bitwiseXor),
     .bitwiseXNor(bitwiseXNor)
    );
   // User logic ends 
```

Also update the register write logic... It will be an always block with a case statement which handles the slv_reg0..7 registers. The cases for slv_reg_wren enabled writing slv_reg2..7 should be removed and replaced with writes every cycle as shown: 

![AXI4 Peripheral Edit Write Side](../images/axi4_peripheral_edit_write_side.png)

### Package your AXI IP

Once these steps are complete:

* Do a basic simulation compile to make sure you have no syntax errors.
* Go back to the "Package IP" tab in the editor window.
* Click on "File Groups" and make sure the source code for your IP is included.
* Make sure all the categories are updated (all checkboxes should be green) and click "Review and Package".
* Click "Re-Package IP"

![AXI Packager Step](../images/axi4_package_ip_simple_logic_module.png)


### Instantiating the new AXI IP in an FPGA

Now start the build of the FPGA overlay: 

* Start a new project named "SimpleLogicModuleFPGAProject"... create it as an RTL project and don't include any IP.  
* Create a new Block Design called "SimpleLogicModuleFPGADesign".
* Select "Project Manager"->"Settings" and add the ip_repo from the prior step. 

Now instance two items using the + button:

* The "SimpleLogicModuleAXIIP" (if it can't be found make sure the proper ip_repo was added). 
* The Zynq 7000 Instance.
   
The diagram window should look like this: 

![Zynq and Simple Logic Module](../images/zynq_and_simple_logic_module.png)

Click on both "Run Block Automation" and "Run Connection Automation". 

Select the top level Design file and click Flow->Create Design Wrapper (the compiler needs to compile RTL, not board files). Then select "Generate Bitstream" to generate a bitstream (the tool will need to go through synthesis and placement steps first). 


## Running the IP on the TUL board

### Collect Bit and HWH Files

Once the FPGA Image is built, collect the following:

* Locate the .hwh file (it will be named after the project-- such as SimpleLogicModuleFPGAProject.hwh). Copy it to the PYNQ board as /home/xilinx/pynq/overlays/SimpleLogicModule/SimpleLogicModule.hwh. 
* Locate the .bit file (it will be named after the wrapper-- such as SimpleLogicModule_wrapper.bit). Copy it to the PYNQ board as /home/xilinx/pynq/overlays/SimpleLogicModule/SimpleLogicModule.bit. 


### Test in Python

Once that is done, start python3 and execute:

```Python
    from pynq import Overlay
    overlay = Overlay('SimpleLogicModule.bit');
    # Make sure this is in    pynq/overlays/SimpleLogicModule/SimpleLogicModule.bit
    # and the hwh file is in  pynq/overlays/SimpleLogicModule/SimpleLogicModule.hwh
    
    a=890
    b=743
    overlay.SimpleLogicModuleAXI_0.write(0x00,a)
    overlay.SimpleLogicModuleAXI_0.write(0x04,b)
    print (" a +    b Expected " + str(a+b)    + " Actual " + str(overlay.SimpleLogicModuleAXI_0.read(0x08)))
    print (" a -    b Expected " + str(a-b)    + " Actual " + str(overlay.SimpleLogicModuleAXI_0.read(0x0c)))
    print (" a and  b Expected " + str(a&b)    + " Actual " + str(overlay.SimpleLogicModuleAXI_0.read(0x10)))
    print (" a or   b Expected " + str(a|b)    + " Actual " + str(overlay.SimpleLogicModuleAXI_0.read(0x14)))
    print (" a xor  b Expected " + str(a^b)    + " Actual " + str(overlay.SimpleLogicModuleAXI_0.read(0x18)))
    print (" a xnor b Expected " + str((2**32) + ~(a^b)) 
                                               + " Actual " + str(overlay.SimpleLogicModuleAXI_0.read(0x1C)))
```

## GOTCHAS

* Be sure IP of all source files (including your module) is exported when you do AXI peripheral packaging. See: https://forums.xilinx.com/t5/Design-Entry/Can-t-get-custom-IP-with-FIFO-to-synthesize-in-the-top-level/m-p/564966 Otherwise the FPGA Implementation step will not be able to find the proper IP.
* Be sure to make a design wrapper HDL out of the top level. Use "Create Design Wrapper" by right-clicking on the top level of the FPGAImplementation. The bitstream program can''t make a file out of board descriptions- it needs the HDLs.
* Don't put slv_reg2-7 assignments into the case statement for the AXI write. You will end up needing to initiate a ghost write cycle to those addresses to make the write happen.
* If you receive compile errors, check the Library Path in the Source File Properties. If it is blank it should  be set to xil_defaultlib. 

## References

1. https://www.xilinx.com/video/hardware/creating-an-axi-peripheral-in-vivado.html  This video walks you through the basics of creating an AXI peripheral.
2. https://www.xilinx.com/support/documentation/ip_documentation/axi_ref_guide/latest/ug1037-vivado-axi-reference-guide.pdf